In [1]:
from openai import OpenAI

In [2]:
from openai import OpenAI
 
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [3]:
from elasticsearch import Elasticsearch

In [5]:
es_client = Elasticsearch('http://localhost:9200')
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "description": {"type": "text"},
            "title": {"type": "keyword"}
        }
    }
}
 
index_name = "kolokvijum1" #database name maybe ?
 
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'kolokvijum1'})

In [6]:
from tqdm.auto import tqdm

In [7]:
import json

In [8]:
with open('kolokvijum.json', 'rt') as f_in:
    titles = json.load(f_in)

In [9]:
for title in tqdm(titles):
    es_client.index(index=index_name,document=title) # final init of elastiserach

  0%|          | 0/122 [00:00<?, ?it/s]

In [10]:
def elastic_search(query):
    serach_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["description^3"],
                        "type": "best_fields"
                    }
                }
               
            }
        }
    }
    response = es_client.search(index=index_name,body=serach_query)
       
    result_doc = []
   
    for hit in response['hits']['hits']:  
        result_doc.append(hit['_source'])
    return result_doc

In [20]:
def build_prompt(query,search_results):
    prompt_template ="""
You are user manuel interpreter. Answer the QUESTION base on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.If CONTEXT doesn't contain  the answer output NONE
 
 
QUESTION: {question}
 
CONTEXT: {context}
""".strip()
 
    context  = ""
    for doc in search_results:
        context = context + f"title: {doc['title']}\ndescirption: {doc['description']}\n\n"
   
    prompt = prompt_template.format(question=query,context= context).strip()
    print(prompt)
    return prompt
 
def llm(prompt):
    response = client.chat.completions.create(
        model='gemma2:2b',
        #model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
   
    return response.choices[0].message.content

In [12]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer


In [13]:
print(rag('Sta je SQL?'))

NONE 



In [21]:
import threading
import time

def loader():
    with tqdm(
        desc="Waiting for LLM", 
        ncols=100, 
        bar_format="{l_bar}{bar} | {elapsed}"
    ) as pbar:
        while not stop_loading:
            time.sleep(0.1)  # Adjust the speed if necessary
            pbar.update(1)

# Start the loader in a separate thread
stop_loading = False
t = threading.Thread(target=loader)
t.start()

# Call the LLM or any other long-running task
response = rag('Metodologija REST aplikacije?')

# Stop the loader
stop_loading = True
t.join()

# Handle the response
print(response)

Waiting for LLM: |                                                                           | 00:00

You are user manuel interpreter. Answer the QUESTION base on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.If CONTEXT doesn't contain  the answer output NONE
 
 
QUESTION: Metodologija REST aplikacije?
 
CONTEXT: title: Dizajn REST aplikacije / metodologija
descirption: Dizajn REST aplikacije / metodologija
1. Identiﬁkovanje resursa koji treba da su vidljivi 
kao servis (npr. godišnji izveštaji, katalozi 
knjiga, porudžbine…) 
2. Modelovanje relacija između resursa kao 
hiperlinkova koje je moguće slediti kako bi se 
dobilo više detalja (ili kako bi se izvela 
promena stanja resursa) 
3. Deiﬁnisati „lepe“ URI-je za adresiranje resursa 
4. Razumeti smisao izvršavanja GET, POST, PUT, 
DELETE zahteva na svaki od resursa (i sa li su 
svi i dozvoljeni za svaki resurs) 
5. Dizajnirati i dokumentovati reprezentaciju 
resursa (može biti više) 
6. Implementirati i postaviti na web server 
7. Testirati (browser, PostMen…) 


title: Šta je REST

Waiting for LLM: |                                                                           | 00:30

REST aplikacija se oslanja na temeljna principe:

1. **Identifikacija resursa:** Identifikacija resursa treba da budu vidljivi kao servis (npr. godišnja izveštaji, katalozi knjiga...). 
2. **Pojedine odgovore:** Modeliranje relacija između resursi kao hiperlinkova koje je moguće slediti kako bi se dobili više detalja (ili kako bi se izvela promjena stanja).
3. **Lekcije za URI-je:** "Lepe" URI-je za adresiranje resursa treba da budu određen.  
4. **Specifikacije zahteva:** Razumeti smisao izvršavanja GET, POST, PUT i DELETE zahteva na svaki od resursa (i sa li su svi i dozvoljeni za svaki resur). 
5. **Representacija resursi:** Reprezentancije resursi treba da budu definisane (može biti više).  
6. **Implemetasija i postavke web-servera:** Implementirati i postaviti na web server. 
7. **Testiranja:**  Testiranje aplikacije  (browser, PostMen...)





